# Single View of Customer

The `svoc` package performs record linkage between two dataframes.  
The objective is to link each record from a _benchmark dataframe_ to at least one record from a second dataframe, referred to as the _input dataframe_.

The script takes as input:
- Two dataframes, benchmark and input (see [Data](#data));
- A [configuration file](#configuration-file), either in `.yaml` or `.env` format.

The record linkage process consists of four main steps:
1. Data preparation;
2. [Features calculation](#features-calculation);
3. Automatic matching;
4. Probabilistic matching.

## Data

Record linkage is performed by comparing the similarity of fields across dataframes.  
The current implementation requires the data to contain the following fields:

| Field        | Content                                      | Example                              |
|:-------------|:---------------------------------------------|:-------------------------------------|
| ID           | Row ID that uniquely identifies each record  | c9425868-0cd2-48eb-9050-8431d9832838 |
| OUTLET_NAME  | Name of the outlet                           | The Golden Fleece                    |
| ADDRESS      | Address of the outlet                        | 9 Queen St, London                   |
| POSTCODE     | Postal code of the outlet                    | EC4N1SP                              |

The data may use different column names; in that case, the corresponding column mappings must be specified in the [configuration file](#configuration-file).

## Configuration file

The `/config` folder contains the .yaml configuration file.  
In the configuration file, you can set the following parameters:

| Parameter               | Description                                                                 | Default value                               | Example                                   |
|:------------------------|:-----------------------------------------------------------------------------|:--------------------------------------------|:------------------------------------------|
| DATA_DIR                | Directory where the benchmark and input `.csv` files are stored              | `./` (current directory)                    | `./data`                                  |
| BENCHMARK_DATA_FILENAME | Name of the `.csv` file containing the benchmark data                         | —                                          | `benchmark_data.csv`                      |
| INPUT_DATA_FILENAME     | Name of the `.csv` file containing the input data                             | —                                          | `input_data.csv`                          |
| BENCHMARK_DATATABLE     | Name of the SQL table containing the benchmark data                           | —                                          | `data.benchmarkdata`                      |
| INPUT_DATATABLE         | Name of the SQL table containing the input data                               | —                                          | `data.inputdatatable`                    |
| BENCHMARK_COLUMNS       | Mapping of required fields to benchmark data column names                     | `{ID, OUTLET_NAME, ADDRESS, POSTCODE}`     | see example below                         |
| INPUT_COLUMNS           | Mapping of required fields to input data column names                         | `{ID, OUTLET_NAME, ADDRESS, POSTCODE}`     | see example below                         |
| MODELS_DIR              | Directory where supervised models are stored                                  | `./models`                                 | `./new_models`                            |
| N_MATCHES               | Maximum number of matches (from input data) per benchmark record              | `3`                                        | `1`                                      |
| BLOCK_COL               | Column used for blocking (matches must share the same value)                  | `'POSTCODE'`                               | _Other values not currently supported_   |

#### _Example_

We want to find a match for each record in the SAP data stored in the CSV file `./data/HUK_sap_data.csv`.  
Possible matches are searched within the Bowimi data stored in `./data/HUK_bowimi_data.csv`.

Since the column names differ from the default ones (see the table above or the [Data](#data) section), they must be explicitly mapped in the configuration file:

```yaml
DATA_DIR: "./data"

BENCHMARK_DATA_FILENAME: "HUK_sap_data.csv"
BENCHMARK_COLUMNS:
  ID: "SapCode"
  OUTLET_NAME: "OutletName"
  POSTCODE: "OutletPostcode"
  ADDRESS: "OutletAddress"

INPUT_DATA_FILENAME: "HUK_bowimi_data.csv"
INPUT_COLUMNS:
  ID: "BowimiId"
  OUTLET_NAME: "OutletName"
  POSTCODE: "OutletPostCode"
  ADDRESS: "OutletAddress"

```
If the data are stored in SQL tables, they can be imported by specifying the corresponding `*_DATATABLE` parameters instead of the `*_FILENAME` ones.
```yaml
BENCHMARK_FILENAME: "rl_data.huk_sap_table"
BENCHMARK_COLUMNS:
  ID: 'SapCode'
  OUTLET_NAME: 'OutletName'
  POSTCODE: 'OutletPostcode'
  ADDRESS: 'OutletAddress'

INPUT_FILENAME: "rl_data.huk_bowimi_table"
INPUT_COLUMNS:
  ID: 'BowimiId'
  OUTLET_NAME: 'OutletName'
  POSTCODE: 'OutletPostCode'
  ADDRESS: 'OutletAddress'
```

If a parameter is not specified in the configuration file, the default value will be used.

### Environment variables

Instead of using a `.yaml` file, parameters can be set as environment variables in a `.env` file.  
In this case:

- Parameter names must be prefixed with `SVOC_`;
- Nested parameters must be specified using a double underscore (`__`) as a separator.

#### _Example_

```env
SVOC_INPUT_FILENAME="rl_data.huk_bowimi_table"
SVOC_BENCHMARK_COLUMNS__ID="SapCode"
SVOC_BENCHMARK_COLUMNS__OUTLET_NAME="OutletName"
# ...
```

### Importing the settings

After defining the parameters, they can be loaded into the script using 
```python 
from svoc.settings import get_settings()
settings = get_settings()   # Load from .env
# or
settings = get_settings("./config/settings.yaml") # Load from a .yaml file 
``` 
 

## Data Preparation

## Features Calculation

Record linkage is performed by comparing the fields of two different dataframes and matching those records whose field values are most similar.

To assess the similarity between two fields, the algorithm relies on the Python
[recordlinkage](https://recordlinkage.readthedocs.io/en/latest/) package, which provides classes and methods to compute several distance measures, referred to as _features_.

The similarity between two strings is represented by a numerical value between 0 (no similarity) and 1 (maximum similarity).  
The `recordlinkage` package offers multiple string distance metrics; the `svoc` package uses the following:

- _Jaro–Winkler_ distance;
- _Levenshtein_ distance;
- _Q-gram_ distance;
- _Cosine_ distance.

In addition, the package includes the following custom measures:

- _Exact_: equal to 1 if the strings are exactly the same, 0 otherwise;
- _Substring_: equal to 1 if one string is entirely contained within the other, 0 otherwise;
- _Word inclusion_: equal to 1 if all the words in one string are contained in the other, 0 otherwise.

All available measures are defined in the `DISTANCES` constant:

```python
from svoc.constants import DISTANCES
DISTANCES
```
This is a list of `Distance()` instances, defines as follows:
``` python
from svoc.constants import Distance
Distance(
    col_name= 'OUTLET_NAME',          # Field name
    method= 'cosine',                 # Distance metric used
    label= 'outlet_name_cosine'       # Feature label
)
```

The following table contains the features currently calculated.

| Field Name        | Method          | Label                                |
|:------------------|:----------------|:-------------------------------------|
| OUTLET_NAME       | Cosine          | outlet_name_cosine                   |
|                   | Jarowinkler     | outlet_name_jarowinkler              |
|                   | Levenshtein     | outlet_name_lenvenshtein             |
|                   | QGram           | outlet_name_qgram                    |
|                   | Exact           | outlet_name                          |
|                   | Substring       | outlet_name_in                       |
|                   | Word Inclusion  | outlet_name_in2                      |
| OUTLET_NAME_CLEAN | Cosine          | outlet_name_clean_cosine             |
|                   | Jarowinkler     | outlet_name_clean_jarowinkler        |
|                   | Levenshtein     | outlet_name_clean_lenvenshtein       |
|                   | QGram           | outlet_name_clean_qgram              |
|                   | Exact           | outlet_name_clean                    |
|                   | Substring       | outlet_name_clean_in                 |
|                   | Word Inclusion  | outlet_name_clean_in2                |
| ADDRESS           | Cosine          | address_cosine                       |
|                   | Jarowinkler     | address_jarowinkler                  |
|                   | Levenshtein     | address_lenvenshtein                 |
|                   | QGram           | address_qgram                        |
|                   | Exact           | address                              |
|                   | Substring       | address_in                           |
|                   | Word Inclusion  | address_in2                          |
| ADDRESS_CLEAN     | Cosine          | address_clean_cosine                 |
|                   | Jarowinkler     | address_clean_jarowinkler            |
|                   | Levenshtein     | address_clean_lenvenshtein           |
|                   | QGram           | address_clean_qgram                  |
|                   | Exact           | address_clean                        |
|                   | Substring       | address_clean_in                     |
|                   | Word Inclusion  | address_clean_in2                    |
